# Custom Chatbot Project

I selected the Food_Scrap_Drop-Off_Locations_in_NYC.csv dataset because it contains real-world, location-specific information that cannot be easily inferred by the base OpenAI model. The chatbot will act as a “NYC Composting Assistant,” helping users find Locations where NYC residents can drop off their food scraps to be composted.

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [49]:
import pandas as pd

In [50]:
df = pd.read_csv("data/Food_Scrap_Drop-Off_Locations_in_NYC.csv")
print(df.head())

     Borough                  NTAName  \
0   Brooklyn                Bay Ridge   
1  Manhattan  East Midtown-Turtle Bay   
2  Manhattan           Hell's Kitchen   
3  Manhattan  East Midtown-Turtle Bay   
4  Manhattan     Tribeca-Civic Center   

                                     SiteName  \
0              4th Avenue Presbyterian Church   
1          Dag Hammarskjold Plaza Greenmarket   
2  Hudson River Park's Pier 84 at W. 44th St.   
3                    58th Street Library FSDO   
4                         Tribeca Greenmarket   

                               SiteAddr                       Hosted_By  \
0   6753 4th Avenue, Brooklyn, NY 11220  4th Avenue Presbyterian Church   
1                   E 47th St & 2nd Ave                         GrowNYC   
2  Pier 84 at W. 44th St. near dog park      Staff at Hudson River Park   
3                  127 East 58th Street                         GrowNYC   
4              Greenwich St. & Duane St                         GrowNYC   

   Open

In [51]:
df["text"] = (
    "Borough: " + df["Borough"].astype(str)+
    " | Site: " + df["SiteName"].astype(str) +
    " | SiteAddress: " + df["SiteAddr"].astype(str) +
    " | OpenMonth: " + df["Open_Month"].astype(str) +
    " | Location: " + df["Location Point"].astype(str) +
    " | Hours: " + df["Day_Hours"].astype(str) +
    " | Notes: " + df["Notes"].astype(str)
)

In [65]:
# Keep only the text column
df = df[["text"]]

# Drop empty characters
df = df[df["text"].str.strip() != ""].dropna(subset=["text"])

# Strip extra whitespace
df["text"] = df["text"].str.strip()

# Reset index
df = df.reset_index(drop=True)

# Ensure at least 20 rows
print("Number of rows after cleaning:", len(df))
print(df.head())

# Verify at least 20 rows
#print("Number of rows:", len(df))
#print(df.head())

Number of rows after cleaning: 591
                                                text
0  Borough: Brooklyn | Site: 4th Avenue Presbyter...
1  Borough: Manhattan | Site: Dag Hammarskjold Pl...
2  Borough: Manhattan | Site: Hudson River Park's...
3  Borough: Manhattan | Site: 58th Street Library...
4  Borough: Manhattan | Site: Tribeca Greenmarket...


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [53]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = ""

In [56]:
def custom_query(query, df):
    context = " ".join(df["text"].head(3).tolist())
    
    prompt = f"""
    You are a helpful chatbot. Use the following dataset information if relevant:

    {context}

    Now answer the user’s question: {query}
    """
    
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",  
        prompt=prompt,
        max_tokens=200,
        temperature=0.5
    )
    
    return response["choices"][0]["text"].strip()

print("Welcome to the NYC Composting Assistant! Type 'exit' to quit.\n")

while True:
    user_input = input("Enter your question: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    
    answer = custom_query(user_input, df)
    print("\nAnswer:", answer)
    print("-"*80)
    
# Example
#print(custom_query("Where can I drop off compost in Brooklyn?", df))


Welcome to the NYC Composting Assistant! Type 'exit' to quit.

Enter your question: where can i drop off compost in union square

Answer: There are several compost drop-off sites in Union Square, including the Union Square Greenmarket and the Union Square Park Recycling Center. These sites are open year round and accept compost every day during their operating hours. However, please note that these sites do not accept meat, bones, or dairy.
--------------------------------------------------------------------------------
Enter your question: xexit

Answer: I'm sorry, I do not understand your question. Can you please clarify or ask a different question?
--------------------------------------------------------------------------------
Enter your question: exit
Goodbye!


In [58]:
# ---- Basic Completion (no dataset) ----
def basic_query(query):
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",   # completion model
        prompt=query,
        max_tokens=200,
        temperature=0.5
    )
    return response["choices"][0]["text"].strip()

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [59]:
questions = [
    "Where can I drop off compost in Brooklyn?"
]

In [60]:
for q in questions:
    print("="*80)
    print(f"QUESTION: {q}\n")

QUESTION: Where can I drop off compost in Brooklyn?



In [61]:
print("🔹 Basic Completion (no dataset):")
print(basic_query(q))
print("\n🔹 Custom Completion (with dataset):")
print(custom_query(q, df))

🔹 Basic Completion (no dataset):
1. McCarren Park Greenmarket Compost Drop-off: Located at the corner of Driggs Avenue and Lorimer Street, this drop-off site is open on Saturdays from 8am-2pm.

2. Brooklyn Borough Hall Greenmarket Compost Drop-off: This drop-off site is open on Saturdays from 8am-2pm, located at the corner of Court Street and Montague Street.

3. Red Hook Community Farm Compost Drop-off: This drop-off site is open on Saturdays from 10am-2pm, located at 580 Columbia Street.

4. Prospect Park Audubon Center Compost Drop-off: Open on Saturdays from 8am-2pm, this drop-off site is located at 101 East Drive in Prospect Park.

5. Brooklyn Botanic Garden Compost Drop-off: This drop-off site is open on Saturdays from 8am-2pm, located at 1000 Washington Avenue.

6. Gowanus Canal Conservancy Compost

🔹 Custom Completion (with dataset):
You can drop off compost in Brooklyn at the 4th Avenue Presbyterian Church located at 6753 4th Avenue. It is open year round and accepts compost e

### Question 2

In [62]:
questions = [
    "What are the hours for the East New York drop-off site?"
]

In [63]:
for q in questions:
    print("="*80)
    print(f"QUESTION: {q}\n")

QUESTION: What are the hours for the East New York drop-off site?



In [64]:
print("🔹 Basic Completion (no dataset):")
print(basic_query(q))
print("\n🔹 Custom Completion (with dataset):")
print(custom_query(q, df))

🔹 Basic Completion (no dataset):
The hours for the East New York drop-off site vary depending on the specific location. It is best to contact the site directly or check their website for their hours of operation.

🔹 Custom Completion (with dataset):
The East New York drop-off site, located at 4th Avenue Presbyterian Church, is open year round from dawn until dusk every day.
